In [45]:
%matplotlib inline
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

In [46]:
df_R_amaral = pd.read_excel('Base de Dados R. Amaral Office.xlsx')
df_R_amaral 

,id_pedido,nome_cliente,modo_envio,estado,vendas,quantidade,desconto,lucro,segmento,região,...,dia_pedido,dia_semana_pedido,mes_pedido,ano_pedido,dia_envio,mes_envio,ano_envio,data_pedido,mes_ano,tempo_preparacao
0,1,Muhammed MacIntyre,Expresso,Paraná,825.174,9,0.30,-117.8820,Corporativo,Sul,...,4,Domingo,9,2016,6,9,2016,42617,42617,2
1,2,Ruben Dartt,Normal,São Paulo,411.332,4,0.15,-4.8392,Consumidor,Sudeste,...,5,Sexta-feira,9,2014,9,9,2014,41887,41887,4
2,3,Liz Pelletier,Mesmo Dia,São Paulo,411.332,4,0.15,-4.8392,Home Office,Sudeste,...,28,Quinta-feira,11,2013,28,11,2013,41606,41606,0
3,4,Liz Pelletier,Expresso,Amapá,241.960,2,0.00,33.8744,Consumidor,Norte,...,30,Segunda-feira,5,2016,31,5,2016,42520,42520,1
4,5,Liz Pelletier,Normal,Sergipe,341.960,2,0.00,78.6508,Home Office,Nordeste,...,31,Terça-feira,12,2013,7,1,2014,41639,41639,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,9990,Max Jones,Expresso,Minas Gerais,1097.544,7,0.20,123.4737,Corporativo,Sudeste,...,9,Quarta-feira,12,2015,11,12,2015,42347,42347,2
9990,9991,Kristina Nunn,Expresso,Mato Grosso,979.950,5,0.00,284.1855,Consumidor,Centro-Oeste,...,2,Sexta-feira,12,2016,4,12,2016,42706,42706,2
9991,9992,Kristina Nunn,Normal,Pernambuco,235.188,2,0.40,-43.1178,Corporativo,Nordeste,...,12,Domingo,1,2014,17,1,2014,41651,41651,5
9992,9993,Sean Miller,Expresso,Alagoas,391.980,2,0.00,113.6742,Consumidor,Nordeste,...,22,Quinta-feira,9,2016,24,9,2016,42635,42635,2


## Tratamento de dados

#### Identificação de problemas:



In [47]:
df_R_amaral.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id_pedido          9994 non-null   int64  
 1   nome_cliente       9994 non-null   object 
 2   modo_envio         9994 non-null   object 
 3   estado             9994 non-null   object 
 4   vendas             9994 non-null   float64
 5   quantidade         9994 non-null   int64  
 6   desconto           9994 non-null   float64
 7   lucro              9994 non-null   float64
 8   segmento           9994 non-null   object 
 9   região             9994 non-null   object 
 10  subcategoria       9994 non-null   object 
 11  categoria          9994 non-null   object 
 12  dia_pedido         9994 non-null   int64  
 13  dia_semana_pedido  9994 non-null   object 
 14  mes_pedido         9994 non-null   int64  
 15  ano_pedido         9994 non-null   int64  
 16  dia_envio          9994 

In [48]:
quantidade_nans = df_R_amaral.isna().sum()
print(quantidade_nans)

id_pedido            0
nome_cliente         0
modo_envio           0
estado               0
vendas               0
quantidade           0
desconto             0
lucro                0
segmento             0
região               0
subcategoria         0
categoria            0
dia_pedido           0
dia_semana_pedido    0
mes_pedido           0
ano_pedido           0
dia_envio            0
mes_envio            0
ano_envio            0
data_pedido          0
mes_ano              0
tempo_preparacao     0
dtype: int64


In [49]:
dados_duplicados = df_R_amaral.duplicated().sum()
print(dados_duplicados)

0


In [50]:
df_R_amaral.dtypes


id_pedido              int64
nome_cliente          object
modo_envio            object
estado                object
vendas               float64
quantidade             int64
desconto             float64
lucro                float64
segmento              object
região                object
subcategoria          object
categoria             object
dia_pedido             int64
dia_semana_pedido     object
mes_pedido             int64
ano_pedido             int64
dia_envio              int64
mes_envio              int64
ano_envio              int64
data_pedido            int64
mes_ano                int64
tempo_preparacao       int64
dtype: object

#### Limpeza de dados

Como não foram identificados valores nulos ou duplicados, vamos apenas padronizar as categorias e substituir outliners

In [51]:
for coluna in df_R_amaral.select_dtypes(include=["object", "category"]).columns:
    df_R_amaral[coluna] = df_R_amaral[coluna].astype(str).str.strip()

In [52]:
colunas_categoricas = [
    'modo_envio', 'estado', 'segmento', 'região',
    'subcategoria', 'categoria', 'dia_semana_pedido'
]

df_R_amaral[colunas_categoricas] = df_R_amaral[colunas_categoricas].astype('category')

df_R_amaral.dtypes

id_pedido               int64
nome_cliente           object
modo_envio           category
estado               category
vendas                float64
quantidade              int64
desconto              float64
lucro                 float64
segmento             category
região               category
subcategoria         category
categoria            category
dia_pedido              int64
dia_semana_pedido    category
mes_pedido              int64
ano_pedido              int64
dia_envio               int64
mes_envio               int64
ano_envio               int64
data_pedido             int64
mes_ano                 int64
tempo_preparacao        int64
dtype: object

In [53]:
df_R_amaral.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   id_pedido          9994 non-null   int64   
 1   nome_cliente       9994 non-null   object  
 2   modo_envio         9994 non-null   category
 3   estado             9994 non-null   category
 4   vendas             9994 non-null   float64 
 5   quantidade         9994 non-null   int64   
 6   desconto           9994 non-null   float64 
 7   lucro              9994 non-null   float64 
 8   segmento           9994 non-null   category
 9   região             9994 non-null   category
 10  subcategoria       9994 non-null   category
 11  categoria          9994 non-null   category
 12  dia_pedido         9994 non-null   int64   
 13  dia_semana_pedido  9994 non-null   category
 14  mes_pedido         9994 non-null   int64   
 15  ano_pedido         9994 non-null   int64   
 16  dia_en

## Verificando valores atribuidos a cada coluna

In [54]:
for coluna in df_R_amaral.select_dtypes(include="category").columns:
    print(f"Valores únicos da coluna '{coluna}':")
    print(df_R_amaral[coluna].unique())
    print('-' * 50)


Valores únicos da coluna 'modo_envio':
['Expresso', 'Normal', 'Mesmo Dia', 'Econômico']
Categories (4, object): ['Econômico', 'Expresso', 'Mesmo Dia', 'Normal']
--------------------------------------------------
Valores únicos da coluna 'estado':
['Paraná', 'São Paulo', 'Amapá', 'Sergipe', 'Rio de Janeiro', ..., 'Distrito Federal', 'Tocantins', 'Paraíba', 'Rondônia', 'Acre']
Length: 27
Categories (27, object): ['Acre', 'Alagoas', 'Amapá', 'Amazonas', ..., 'Santa Catarina', 'Sergipe', 'São Paulo', 'Tocantins']
--------------------------------------------------
Valores únicos da coluna 'segmento':
['Corporativo', 'Consumidor', 'Home Office']
Categories (3, object): ['Consumidor', 'Corporativo', 'Home Office']
--------------------------------------------------
Valores únicos da coluna 'região':
['Sul', 'Sudeste', 'Norte', 'Nordeste', 'Centro-Oeste']
Categories (5, object): ['Centro-Oeste', 'Nordeste', 'Norte', 'Sudeste', 'Sul']
--------------------------------------------------
Valores ún

## Análise descritiva das variaveis numéricas

In [55]:
colunas_analisar = ['vendas', 'quantidade', 'desconto', 'lucro', 'tempo_preparacao']

for coluna in colunas_analisar:
    print(f"Estatísticas da coluna '{coluna}':")
    print(df_R_amaral[coluna].describe().round(3))
    print('-' * 50)

Estatísticas da coluna 'vendas':


count     9994.000
mean       229.858
std        623.245
min          0.444
25%         17.280
50%         54.490
75%        209.940
max      22638.480
Name: vendas, dtype: float64
--------------------------------------------------
Estatísticas da coluna 'quantidade':
count    9994.000
mean        3.790
std         2.225
min         1.000
25%         2.000
50%         3.000
75%         5.000
max        14.000
Name: quantidade, dtype: float64
--------------------------------------------------
Estatísticas da coluna 'desconto':
count    9994.000
mean        0.156
std         0.206
min         0.000
25%         0.000
50%         0.200
75%         0.200
max         0.800
Name: desconto, dtype: float64
--------------------------------------------------
Estatísticas da coluna 'lucro':
count    9994.000
mean       28.657
std       234.260
min     -6599.978
25%         1.729
50%         8.666
75%        29.364
max      8399.976
Name: lucro, dtype: float64
--------------------------------------

# Analise RFM (recência, frequência e monetário)

In [ ]:
import datetime as dt

file_path = 'Base de Dados R. Amaral Office.xlsx'
df_R_amaral = pd.read_excel(file_path, dtype={'data_pedido': float})

df_R_amaral['data_pedido'] = pd.to_datetime('1899-12-30') + pd.to_timedelta(df_R_amaral['data_pedido'], 'D')

data_atual = dt.datetime(2025, 5, 11) #dia que eu fiz o codigo mas podemos mudar para o dia da apresentacao final do case 

rfm = df_R_amaral.groupby('nome_cliente').agg({
    'data_pedido': lambda x: (data_atual - x.max()).days,
    'id_pedido': 'count',
    'vendas': 'sum'
}).reset_index()

rfm.columns = ['Cliente', 'Recencia', 'Frequencia', 'Monetario']

# Definimos que pegaríamos o top 20% de cada quesito para analisarmos
limiar_monetario = rfm['Monetario'].quantile(0.8)
limiar_recencia = rfm['Recencia'].quantile(0.8)
x
clientes_prioritarios = rfm[(rfm['Monetario'] >= limiar_monetario) & (rfm['Recencia'] > 730) ] # Filtrando quem mais gastou e está há mais de 6 meses sem comprar

clientes_prioritarios = clientes_prioritarios.sort_values(by=['Monetario', 'Recencia'], ascending=[False, False])

print("Clientes gastaram bastante, porém e estão há mais tempo sem comprar:")
print(clientes_prioritarios.head(10)) #escolhemos mostrar apenas o top 10 para avaliar se seria bom um atendimento personalizado para entender o que fez com que eles parassem de comprar

Clientes gastaram bastante, porém e estão há mais tempo sem comprar:
              Cliente  Recencia  Frequencia   Monetario
321     Harold Dahlen      3068          36  36521.6425
173      Clytie Kelty      3068          24  26791.6070
423        Karl Brown      3056          37  24383.7630
247        Ed Braxton      3062          46  22441.4620
161  Christy Brittain      3087          30  18250.8500
383        Joe Elijah      3090          12  17862.3020
685       Scott Cohen      3140          16  15587.1690
306  Giulietta Dortch      3056          32  15052.4260
89        Bill Eplett      3097          24  15027.7620
265        Erica Bern      3121          22  14684.0670
